# ESM

In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
from scr.preprocess.data_process import TaskProcess

In [4]:
TaskProcess().sum_file_df

,task,dataset,split,csv_path,fasta_path,pkl_path
0,annotation,scl,balanced,data/annotation/scl/balanced.csv,,
1,structure,secondary_structure,cb513,[data/structure/secondary_structure/secondary_...,,
2,proeng,gb1,two_vs_rest,data/proeng/gb1/two_vs_rest.csv,data/proeng/gb1/5LDE_1.fasta,data/proeng/gb1/two_vs_rest.pkl
3,proeng,gb1,low_vs_high,data/proeng/gb1/low_vs_high.csv,data/proeng/gb1/5LDE_1.fasta,data/proeng/gb1/low_vs_high.pkl
4,proeng,aav,one_vs_many,data/proeng/aav/one_vs_many.csv,data/proeng/aav/P03135.fasta,data/proeng/aav/one_vs_many.pkl
5,proeng,aav,two_vs_many,data/proeng/aav/two_vs_many.csv,data/proeng/aav/P03135.fasta,data/proeng/aav/two_vs_many.pkl
6,proeng,thermo,mixed,data/proeng/thermo/mixed.csv,,


In [3]:
from scr.utils import pickle_load

In [4]:
df = pickle_load("data/proeng/gb1/two_vs_rest.pkl")

In [5]:
df.head()

,sequence,target,set,validation,mut_name,mut_numb
0,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYD...,1.000000,train,NaN,parent,1
1,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGIDGEWTYD...,1.445905,train,NaN,V39I,1
2,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGLDGEWTYD...,1.690164,train,NaN,V39L,1
3,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGMDGEWTYD...,1.170550,train,NaN,V39M,1
4,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVAGEWTYD...,2.401243,train,NaN,D40A,1


In [6]:
df_train = df.loc[(df["set"] == "train") & (df["validation"] != True)]
df_val = df.loc[(df["set"] == "train") & (df["validation"] == True)]
df_test = df.loc[(df["set"] == "test")]

len(df_train), len(df_val), len(df_test), len(df)

(381, 43, 8309, 8733)

In [7]:
from scr.encoding.encoding_classes import ESMEncoder

In [8]:
no_flat_encoder = ESMEncoder(
    encoder_name="esm1b_t33_650M_UR50S",
    embed_layer=0,
).encode(mut_seqs=list(df_val.sequence), mut_names=list(df_val.mut_name))
one_emb = next(no_flat_encoder)
one_emb.shape

Loading esm1b_t33_650M_UR50S using 0 layer embedding


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


No embedding flattening


(43, 265, 1280)

In [9]:
mean_flat_encoder = ESMEncoder(
    encoder_name="esm1b_t33_650M_UR50S",
    embed_layer=0,
).encode(mut_seqs=list(df_val.sequence), mut_names=list(df_val.mut_name),flatten_emb="mean")
one_mean_emb = next(mean_flat_encoder)
one_mean_emb.shape

Loading esm1b_t33_650M_UR50S using 0 layer embedding


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


(43, 1280)

In [10]:
mean_batch_encoder = ESMEncoder(
    encoder_name="esm1b_t33_650M_UR50S",
    embed_layer=0,
).encode(mut_seqs=list(df_val.sequence), mut_names=list(df_val.mut_name), batch_size=4, flatten_emb="mean")
one_mean_emb = next(mean_batch_encoder)

Loading esm1b_t33_650M_UR50S using 0 layer embedding


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main
  0%|          | 0/11 [00:00<?, ?it/s]

In [11]:
one_mean_emb[0].shape, len(one_mean_emb[1])

((4, 1280), 4)